In [2]:
import pandas as pd
import os
import pypdf
import fitz
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings,OpenAIEmbeddings,ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma,MongoDBAtlasVectorSearch
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

True

In [4]:
pdf_file=fitz.open("RagSampleDoc.pdf", filetype="pdf")
text=pdf_file[0].get_text()
print(text)


We are doing a live session understating about RAG. 
This session is being help on teams 
The date of this session is on 17-Aug-2024 
Timing is from 10 AM to 12:00 PM 
Participants are  
Ankit, 
Syed 
Binoy 
 



In [11]:
text_splitter=CharacterTextSplitter(chunk_size=20,chunk_overlap=5)
chunked_text=text_splitter.split_text(text)
print(chunked_text)


['We are doing a live session understating about RAG. \nThis session is being help on teams \nThe date of this session is on 17-Aug-2024 \nTiming is from 10 AM to 12:00 PM \nParticipants are  \nAnkit, \nSyed \nBinoy']


In [12]:
OPENAI_API_KEY=os.environ["OPENAI_API_KEY"]
model_llm = ChatOpenAI(model_name="gpt-4o", temperature=0, api_key=OPENAI_API_KEY)
llm_embeddings=OpenAIEmbeddings(api_key=OPENAI_API_KEY)

In [13]:
chroma_vector_db=Chroma(persist_directory="chroma_vector_db",embedding_function=llm_embeddings)
chroma_vector_db.add_texts(chunked_text,persist=True,llm_embeddings=llm_embeddings)
chroma_vector_db.persist()

c:\Users\syedf\OneDrive\Desktop\github\airimtraining\airimapi\venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(
c:\Users\syedf\OneDrive\Desktop\github\airimtraining\airimapi\venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [14]:
prompt_template="""You are AI assitant that is expert in extracting information from PDFs. You have been given a PDF document that contains information about live session. You need to extract the information from the PDF document and answer the following questions:"
{context}
Question: {question}
"""
PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}
retrievar=chroma_vector_db.as_retriever()
chain=RetrievalQA.from_chain_type(retrievar=retrievar,llm=model_llm,chain_type="stuff",chain_type_kwargs=chain_type_kwargs)
response=chain.invoke("What is the session about?")

TypeError: BaseRetrievalQA.from_chain_type() missing 1 required positional argument: 'llm'